In [1]:
from imutils.object_detection import non_max_suppression
from google.colab.patches import cv2_imshow
from imutils import paths
import tensorflow as tf
import numpy as np
import time
import cv2
import os

In [2]:
!git clone https://github.com/dilhelh/opencv-text-detection.git

Cloning into 'opencv-text-detection'...
remote: Enumerating objects: 11, done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (11/11), done.


In [3]:
!python /content/opencv-text-detection/text_detection.py --image /content/opencv-text-detection/images/lebron_james.jpg \
	--east /content/opencv-text-detection/frozen_east_text_detection.pb

[INFO] loading EAST text detector...
[INFO] text detection took 0.571660 seconds
: cannot connect to X server 


In [4]:
!wget -q https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/releases/download/v0.11.0/coco_text_100.tar.gz
!tar xf coco_text_100.tar.gz

In [5]:
image_list = list(paths.list_images('/content/coco_text_100'))
image_size = 320

In [6]:
def representative_dataset():
    for image_path in image_list:
        image = cv2.imread(image_path)
        image = cv2.resize(image, (image_size, image_size))
        image = image.astype("float32")
        mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
        image -= mean
        image = np.expand_dims(image, axis=0)
        yield [image]

In [13]:
quantization = "int8" #@param ["dr", "int8", "float16"]
model_converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/opencv-text-detection/frozen_east_text_detection.pb', 
    input_arrays=['input_images'],
    output_arrays=['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3'],
    input_shapes={'input_images': [1, 320, 320, 3]}
)

model_converter.optimizations = [tf.lite.Optimize.DEFAULT]

if quantization=="float16":
    model_converter.target_spec.supported_types = [tf.float16]
elif quantization=="int8":
    model_converter.representative_dataset = representative_dataset
    model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    model_converter.inference_input_type = tf.uint8
    model_converter.inference_output_type = tf.uint8

model_tflite = model_converter.convert()
open('TextDetection_Model_{}.tflite'.format(quantization), 'wb').write(model_tflite)

24823088

In [14]:
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

In [15]:
!ls -lh *.tflite
!ls -lh /content/opencv-text-detection/frozen_east_text_detection.pb

-rw-r--r-- 1 root root 24M Jun 13 04:12 TextDetection_Model_dr.tflite
-rw-r--r-- 1 root root 46M Jun 13 04:12 TextDetection_Model_float16.tflite
-rw-r--r-- 1 root root 24M Jun 13 04:14 TextDetection_Model_int8.tflite
-rw-r--r-- 1 root root 93M Jun 13 04:08 /content/opencv-text-detection/frozen_east_text_detection.pb
